# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
#!pip install python-dotenv
#!pip install openai

In [2]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY") 

# check the sever type according to API key
if openai.api_key.startswith("sk-"): # OpenAI
    server_type = "OpenAI"
else:
    server_type = "AzureOpenAI"

# update env info if serer type is AzureOpenAI
if server_type == "AzureOpenAI": # AzureOpenAI
    # Load config values
    with open(r'../azure_config.json') as config_file:
        config_details = json.load(config_file)
        # Set AzureOpenAI API values
        openai.api_type = config_details["OPENAI_API_TYPE"]
        os.environ['OPENAI_API_BASE'] = config_details["OPENAI_API_BASE"]
        openai.api_base=config_details["OPENAI_API_BASE"]
        os.environ["OPENAI_API_VERSION"] = config_details["OPENAI_API_VERSION"]
        openai.api_version=config_details["OPENAI_API_VERSION"]

# check the enironment variable
print(f"sever_type: {server_type} | api_type: {openai.api_type}")

sever_type: AzureOpenAI | api_type: azure


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    if server_type == "AzureOpenAI":
        response = openai.ChatCompletion.create(
            deployment_id="gpt-35-turbo",
            messages=messages,
            temperature=0, 
        )
    else:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
        )
    return response.choices[0].message["content"]

In [4]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [5]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
style = """American English \
in a calm and respectful tone
"""

In [7]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [8]:
response = get_completion(prompt)
print(response)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [9]:
# !pip install --upgrade langchain

### Model

In [10]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

# construct chat function according to API key format
if server_type == "OpenAI":
    chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.9)
else:
    chat=AzureChatOpenAI(deployment_name="gpt-35-turbo",model_name="gpt-35-turbo", temperature=0.9)

### Prompt template

In [11]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
print(template_string)

Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```



In [12]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
# print(prompt_template)
for prompt_info in prompt_template:
    print(prompt_info)

('lc_kwargs', {'input_variables': ['text', 'style'], 'messages': [HumanMessagePromptTemplate(lc_kwargs={'prompt': PromptTemplate(lc_kwargs={'input_variables': ['style', 'text'], 'template': 'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'}, input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)}, prompt=PromptTemplate(lc_kwargs={'input_variables': ['style', 'text'], 'template': 'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'}, input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True), ad

In [13]:
prompt_template.input_variables

['text', 'style']

In [14]:
prompt_template.messages[0].prompt.template

'Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'

In [15]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [16]:
customer_style = """American English \
in a calm and respectful tone
"""

In [17]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [18]:
# dir(prompt_template)

In [19]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [20]:
customer_messages

[HumanMessage(lc_kwargs={'content': "Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", 'additional_kwargs': {}}, content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, example=False)]

In [21]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [22]:
print(customer_messages[0])

lc_kwargs={'content': "Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", 'additional_kwargs': {}} content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [23]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [24]:
print(customer_response)

lc_kwargs={'content': "I'm really upset that my blender lid came off and splattered smoothie all over my kitchen walls. Adding insult to injury, the warranty doesn't cover the cost of cleaning up the mess. Can you assist me, please?"} content="I'm really upset that my blender lid came off and splattered smoothie all over my kitchen walls. Adding insult to injury, the warranty doesn't cover the cost of cleaning up the mess. Can you assist me, please?" additional_kwargs={} example=False


In [25]:
print(customer_response.content)

I'm really upset that my blender lid came off and splattered smoothie all over my kitchen walls. Adding insult to injury, the warranty doesn't cover the cost of cleaning up the mess. Can you assist me, please?


In [26]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [27]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [28]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [29]:
service_messages

[HumanMessage(lc_kwargs={'content': "Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n", 'additional_kwargs': {}}, content="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n", additional_kwargs={}, example=False)]

In [30]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy, me hearty customer! I be sorry to inform ye that the warranty don't be coverin' the cost of cleanin' yer galley due to yer own actions. 'Twas yer fault for misusin' yer blender by forgettin' to put the lid on before settin' it in motion. Ye be outta luck, matey! Til we meet again!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [31]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [32]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [33]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

lc_kwargs={'input_variables': ['text'], 'messages': [HumanMessagePromptTemplate(lc_kwargs={'prompt': PromptTemplate(lc_kwargs={'input_variables': ['text'], 'template': 'For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'}, input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this informat

In [34]:
# construct chat function according to API key format
if server_type == "OpenAI":
    chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.9)
else:
    chat=AzureChatOpenAI(deployment_name="gpt-35-turbo",model_name="gpt-35-turbo", temperature=0.9)

messages = prompt_template.format_messages(text=customer_review)

response = chat(messages)
print(response.content)

{
"gift": True,
"delivery_days": 2,
"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [35]:
type(response.content)

str

In [36]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
# response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [37]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [38]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [39]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [40]:
format_instructions = output_parser.get_format_instructions()


In [41]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [42]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [43]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [44]:
response = chat(messages)

In [45]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [46]:
type(response.content)

str

In [47]:
output_dict = output_parser.parse(response.content)

In [48]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [49]:
type(output_dict)

dict

In [50]:
output_dict.get('delivery_days')

2